In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv('/Users/kb/Ironhack/Labs/lab-cross-validation/files_for_lab/Customer-Churn.csv')

In [2]:
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
data.shape

(7043, 16)

In [4]:
data['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [5]:
y = data['Churn']
x = data.drop(['Churn'], axis = 1)

In [6]:
num = x.select_dtypes(np.number)
cat = x.select_dtypes(np.object)

In [7]:
encoder = OneHotEncoder(drop='first').fit(cat)
encoded_cat = encoder.transform(cat).toarray()
encoded_cat = pd.DataFrame(encoded_cat)

In [8]:
x = pd.concat([num, encoded_cat], axis = 1)

In [9]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_sm, y_sm = smote.fit_resample(x, y)
y_sm.value_counts()

Yes    5174
No     5174
Name: Churn, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state=0)

In [11]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
model1 = DecisionTreeClassifier()
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression(random_state=0, solver='lbfgs',multi_class='ovr')

In [13]:
import numpy as np
model_pipeline = [model1, model2]
model_names = ['SM Regression Tree', 'SM Logistic Regression']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=10))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'SM Regression Tree': 0.8393255184492299, 'SM Logistic Regression': 0.7749038066048375}


In [14]:
from imblearn.under_sampling import TomekLinks
tl = TomekLinks('majority')
xtl, ytl = tl.fit_resample(x, y)
ytl.value_counts()

No     4716
Yes    1869
Name: Churn, dtype: int64

In [15]:
xtl2, ytl2 = tl.fit_resample(xtl, ytl)
ytl2.value_counts()

No     4579
Yes    1869
Name: Churn, dtype: int64

In [16]:
model_pipeline = [model1, model2]
model_names = ['TK Regression Tree', 'TK Logistic Regression']
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, xtl2, ytl2, cv=10))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'SM Regression Tree': 0.8393255184492299, 'SM Logistic Regression': 0.7749038066048375, 'TK Regression Tree': 0.7968397611825315, 'TK Logistic Regression': 0.809398141460831}


In [17]:
from sklearn.tree import DecisionTreeRegressor
model3 = RandomForestClassifier(max_depth=2, random_state=0)

In [18]:
mean_score = np.mean(cross_val_score(model3, X_train, y_train, cv=10))
scores['SM Random Forest'] = mean_score

In [19]:
print(scores)

{'SM Regression Tree': 0.8393255184492299, 'SM Logistic Regression': 0.7749038066048375, 'TK Regression Tree': 0.7968397611825315, 'TK Logistic Regression': 0.809398141460831, 'SM Random Forest': 0.7613771908875002}
